# Creatring BIDS dataset

## Creating a dataset description

In [33]:
addpath(genpath(fullfile(pwd, '..', 'src')))
addpath(genpath(fullfile(pwd, '..', 'lib')))

In [34]:
cfg.dir.output = fullfile(pwd, 'output');

cfg.task.name = 'test task';

cfg.subject.subjectNb = 1;
cfg.subject.runNb = 1;

cfg.testingDevice = 'mri';

cfg.eyetracker.do = 1;

cfg.verbose = 2;

In [35]:
cfg.bids.datasetDescription.Name = 'my new bids data set';
cfg.bids.datasetDescription.BIDSVersion =  '1.3';
cfg.bids.datasetDescription.License = 'CC-BY';
cfg.bids.datasetDescription.Authors = {'John Wick', 'Napoléon Bonaparte'};
cfg.bids.datasetDescription.Acknowledgements = '';
cfg.bids.datasetDescription.HowToAcknowledge = '';
cfg.bids.datasetDescription.Funding = {''};
cfg.bids.datasetDescription.ReferencesAndLinks = {''};
cfg.bids.datasetDescription.DatasetDOI = '';

[cfg] = createFilename(cfg);



___________________________________________________
___________________________________________________
                                                   
         ___ ___ ___   ___ ___ ___  ___            
        / __| _ \ _ \ | _ )_ _|   \/ __|
       | (__|  _/  _/ | _ \| || |) \__ \
        \___|_| |_|   |___/___|___/|___/
                                                   
Thank you for using the CPP BIDS - version v1.0.0. 

Current list of contributors includes:
 Rémi Gau
 Marco Barilari
 Ceren Battal
 Tomas Lenc


Please cite using the following DOI: 
 https://doi.org/10.5281/zenodo.4007674

For bug report, suggestions or contributions see: 
 https://github.com/cpp-lln-lab/CPP_BIDS

___________________________________________________
___________________________________________________



Data will be saved in this directory:
	/home/remi/github/CPP_BIDS/notebooks/output/source/sub-001/ses-001/func

Data will be saved in this file:
	sub-001_ses-001_task-testTask_run-001_event

In [36]:
createDatasetDescription(cfg)

In [37]:
!tree output

output
└── source
    ├── dataset_description.json
    └── sub-001
        └── ses-001
            └── func

4 directories, 1 file



In [38]:
!cat output/source/dataset_description.json

{
    "Acknowledgements": "",
    "Authors": [
        "John Wick",
        "Napoléon Bonaparte"
    ],
    "BIDSVersion": "1.3",
    "DatasetDOI": "",
    "Funding": [""],
    "HowToAcknowledge": "",
    "License": "CC-BY",
    "Name": "my new bids data set",
    "ReferencesAndLinks": [""]
}


## fMRI

### fMRI specific name suffixes

You can specify some extra information for you fMRI runs as some of this information can be important for filenaming or to add to some JSON files.

In [45]:
cfg.mri.repetitionTime = 2.1;
cfg.suffix.phaseEncodingDirection = 'i-';
cfg.suffix.reconstruction = 'motion correction';
cfg.suffix.echo = '1';
cfg.suffix.acquisition = 'slow';

In [48]:
[cfg] = createFilename(cfg);



___________________________________________________
___________________________________________________
                                                   
         ___ ___ ___   ___ ___ ___  ___            
        / __| _ \ _ \ | _ )_ _|   \/ __|
       | (__|  _/  _/ | _ \| || |) \__ \
        \___|_| |_|   |___/___|___/|___/
                                                   
Thank you for using the CPP BIDS - version v1.0.0. 

Current list of contributors includes:
 Rémi Gau
 Marco Barilari
 Ceren Battal
 Tomas Lenc


Please cite using the following DOI: 
 https://doi.org/10.5281/zenodo.4007674

For bug report, suggestions or contributions see: 
 https://github.com/cpp-lln-lab/CPP_BIDS

___________________________________________________
___________________________________________________



Data will be saved in this directory:
	/home/remi/github/CPP_BIDS/notebooks/output/source/sub-001/ses-001/func

Data will be saved in this file:
	sub-001_ses-001_task-testTask_run-001_event

In [47]:
disp(cfg.fileName.events)

sub-001_ses-001_task-testTask_run-001_events_date-202011171531.tsv


### Saving a `_bold.json` file

There is bare minimum functionality to create a `_bold.json` file for fMRI runs: the information in there wil be that required to pass a BIDS validation but no more. You should REALLY get most of the information for `bold.json` from your favorite DICOM to Nifti converter

This function is there because it can help put in JSON format some of the information that would NOT be available to a converter (usually information about your task).

In [51]:
createJson(cfg, 'func')

In [52]:
!tree output

output
└── source
    ├── dataset_description.json
    └── sub-001
        └── ses-001
            └── func
                ├── sub-001_ses-001_task-testTask_run-001_bold_date-202011171530.json
                └── sub-001_ses-001_task-testTask_run-001_bold_date-202011171531.json

4 directories, 3 files



In [44]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

{
    "Instructions": "",
    "RepetitionTime": [],
    "SliceTiming": [],
    "TaskDescription": "",
    "TaskName": "test Task"
}


### Adding extra info to your `_bold.json`

One thing this function CAN do though is help you save some of the information about your experiment in a json format rather than a `.mat` format. The former being a simple text file you can easily open on any computer.

For example if you have defined some other parameters in another structure.

In [14]:
%% Timing info

% Time between blocs in secs
parameters.timing.IBI = 1.8 * 3; % 8;
% Time between events in secs
parameters.timing.ISI = 0.1;
% Number of seconds before the motion stimuli are presented
parameters.timing.onsetDelay = .1;
% Number of seconds after the end all the stimuli before ending the run
parameters.timing.endDelay = .1;

parameters.timing.eventDuration = 0.8; % second

%% Visual Stimulation

% Speed in visual angles / second
parameters.dot.speed = 30;
% Coherence Level (0-1)
parameters.dot.coherence = 1;
% Number of dots per visual angle square.
parameters.dot.density = .1;
% Dot life time in seconds
parameters.dot.lifeTime = 10;
% proportion of dots killed per frame
parameters.dot.proportionKilledPerFrame = 0;
% Dot Size (dot width) in visual angles.
parameters.dot.size = 1;

In [13]:
createJson(cfg, parameters)

In [15]:
!cat output/source/sub-001/ses-001/func/sub-001*bold_date-*.json

{
    "Instructions": "",
    "RepetitionTime": [],
    "SliceTiming": [],
    "TaskDescription": "",
    "TaskName": "test Task"
}


## Converting your data set from source to raw

Before we do that, let's adding some actual data to the dataset.

In [18]:

logFile.extraColumns = {'target_position', 'target_type'};

logFile = saveEventsFile('init', cfg, logFile);
logFile = saveEventsFile('open', cfg, logFile);

logFile(1,1).onset = 2;
logFile(1,1).trial_type = 'motion_up';
logFile(1,1).duration = 1;
logFile(1,1).target_position = 'left';
logFile(1,1).target_type = 'audio';

logFile(2,1).onset = 6;
logFile(2,1).trial_type = 'static';
logFile(2,1).duration = 1.2;
logFile(2,1).target_position = 'right';
logFile(2,1).target_type = 'visual';

logFile = saveEventsFile('save', cfg, logFile);

saveEventsFile('close', cfg, logFile);

onset	duration	trial_type	target_position	target_type
ans =  2
2.000000	ans =  1
1.000000	motion_up	left	audio	
ans =  6
6.000000	ans =  1.2000
1.200000	static	right	visual	

Data were saved in this file:

/home/remi/github/CPP_BIDS/notebooks/output/source/sub-001/ses-001/func/sub-001_ses-001_task-testTask_run-001_events_date-202011171307.tsv



In [53]:
!tree output

output
└── source
    ├── dataset_description.json
    └── sub-001
        └── ses-001
            └── func
                ├── sub-001_ses-001_task-testTask_run-001_bold_date-202011171530.json
                └── sub-001_ses-001_task-testTask_run-001_bold_date-202011171531.json

4 directories, 3 files



The very basic conversion here mostly entails:
- adding empty README and CHANGES files in the `source` folder
- copying the content of the `source` folder into a `raw` folder
- removing the date suffixes
- compressing `_stim.tsv` into `_stim.tsv.gz`

In [20]:
convertSourceToRaw(cfg)

In [21]:
!tree

.
├── basic_usage.ipynb
├── creating_BIDS_dataset.ipynb
├── output
│   ├── raw
│   │   ├── CHANGES
│   │   ├── dataset_description.json
│   │   ├── README
│   │   └── sub-001
│   │       └── ses-001
│   │           └── func
│   │               ├── sub-001_ses-001_task-testTask_run-001_bold.json
│   │               ├── sub-001_ses-001_task-testTask_run-001_events.json
│   │               └── sub-001_ses-001_task-testTask_run-001_events.tsv
│   └── source
│       ├── CHANGES
│       ├── dataset_description.json
│       ├── README
│       └── sub-001
│           └── ses-001
│               └── func
│                   ├── sub-001_ses-001_task-testTask_run-001_bold_date-202011171307.json
│                   ├── sub-001_ses-001_task-testTask_run-001_events_date-202011171307.json
│                   └── sub-001_ses-001_task-testTask_run-001_events_date-202011171307.tsv
├── README.md
├── saving_arrays.ipynb
├── saving_stim_files.ipynb
└── using_2_logfiles.ipynb

9 directories, 18 files

